In [1]:
class Neuron:
    def __init__(self, bias):
        self.bias = bias
        self.weights = []

    # 傳入input並送進activation function, return output----> a=sigma(z)
    def calculate_output(self, inputs):
        self.inputs = inputs
        self.output = self.activator(self.calculate_total_net_input())
        return self.output
    
    # Input layer -> Hidden layer, 求 z = w1*x1 + w2*x2 + ... + b 
    def calculate_total_net_input(self):
        total = 0
        for i in range(len(self.inputs)):
            total += self.inputs[i] * self.weights[i]
        total += self.bias
        return total

    # Sigmoid函數
    def activator(self, total_net_input):
        return 1 / (1 + math.exp(-total_net_input))

    
    def calculate_pc_on_pz(self, target_output):
        # ∂C/∂z = ∂C/∂y * ∂y/∂z
        pd_errtotal = self.calculate_pc_on_py(target_output)
        pd_out      = self.calculate_py_on_pz()
        return pd_errtotal * pd_out

    # 每一個神經元的誤差: 1/2*(target - predict)^2
    def calculate_error(self, target_output):
        return 0.5 * (target_output - self.output) ** 2

    def calculate_pc_on_py(self, target_output):
        # ∂C/∂y
        return -(target_output - self.output) 

    def calculate_py_on_pz(self):
        # ∂y/∂z
        return self.output * (1 - self.output)

    def activation_input(self, index):
        return self.inputs[index]

In [2]:
class NeuronLayer:
    def __init__(self, num_neurons, bias):
        # 同一層Layer共享參數ｂ
        if bias != 0.0:
            self.bias = bias
        else:
            self.bias = random.random()
        
        # 序列裡收集的是 Neuron物件
        self.neurons = []
        for i in range(num_neurons):
            self.neurons.append(Neuron(self.bias))

    def inspect_NL(self):
        for n in range(len(self.neurons)):
            print(' Neuron', n)
            for w in range(len(self.neurons[n].weights)):
                print('  Weight:', self.neurons[n].weights[w])
            print('  Bias:', self.bias)

    def feed_forward(self, inputs):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.calculate_output(inputs))
        return outputs

    def get_outputs(self):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.output)
        return outputs

In [3]:
class NeuralNetwork:
    LEARNING_RATE = 0.5

    def __init__(self, num_inputs, num_hidden, num_outputs, hidden_layer_weights = None, hidden_layer_bias = None, output_layer_weights = None, output_layer_bias = None):
        self.num_inputs = num_inputs
        
        # hidden layer物件
        self.hidden_layer = NeuronLayer(num_hidden, hidden_layer_bias) 
        # Output layer物件 
        self.output_layer = NeuronLayer(num_outputs, output_layer_bias)

        self.init_weights_from_inputs_to_hidden_layer_neurons(hidden_layer_weights)
        self.init_weights_from_hidden_layer_neurons_to_output_layer_neurons(output_layer_weights)

    def init_weights_from_inputs_to_hidden_layer_neurons(self, hidden_layer_weights):
        weight_num = 0
        for h in range(len(self.hidden_layer.neurons)):
            for i in range(self.num_inputs):
                self.hidden_layer.neurons[h].weights.append(hidden_layer_weights[weight_num])
                weight_num += 1

    def init_weights_from_hidden_layer_neurons_to_output_layer_neurons(self, output_layer_weights):
        weight_num = 0
        for o in range(len(self.output_layer.neurons)):
            for h in range(len(self.hidden_layer.neurons)):
                self.output_layer.neurons[o].weights.append(output_layer_weights[weight_num])
                weight_num += 1

    def feed_forward(self, inputs):
        hidden_layer_outputs = self.hidden_layer.feed_forward(inputs)
        return self.output_layer.feed_forward(hidden_layer_outputs)

    def feed_backward(self, targets_outputs):
        # Case1: Output_Layer 求 ∂C/∂z 
        pd_C_on_z_OutputLayer = [None] * len(self.output_layer.neurons)
        for o in range(len(self.output_layer.neurons)):
            # ∂C/∂z = ∂C/∂y * ∂y/∂z
            pd_C_on_z_OutputLayer[o] = self.output_layer.neurons[o].calculate_pc_on_pz(targets_outputs[o])

        # Case2: Hidden_Layer 求 ∂C/∂z
        pd_C_on_z_NotOutputLayer = [None] * len(self.hidden_layer.neurons)
        for h in range(len(self.hidden_layer.neurons)):
            sum_w_mul_pc_on_pz = 0
            for o in range(len(self.output_layer.neurons)):
                # ∂C/∂Out_h = (∂C/∂Out_o) * (∂Out_o/∂Net_o) *  [(∂Net_o/∂Out_h)]
                # w3 * ∂C/∂z' + w4 * ∂C/∂z"
                pd_C_on_Outh = pd_C_on_z_OutputLayer[o] * self.output_layer.neurons[o].weights[h]
                # Σ C/∂Out_h
                sum_w_mul_pc_on_pz += pd_C_on_Outh
            # (∂Out_h/∂Net_h)
            pd_outh_neth = self.hidden_layer.neurons[h].calculate_py_on_pz()
            # ∂C/∂Net_h = (∂C/∂Out_h) * (∂Out_h/∂Net_h)
            pd_C_on_z_NotOutputLayer[h] = sum_w_mul_pc_on_pz * pd_outh_neth

        # 3. 更新输出層權重系数
        for o in range(len(self.output_layer.neurons)):
            for w_h2o in range(len(self.output_layer.neurons[o].weights)):
                # ∂z/∂w = Out_h
                Out_h = self.output_layer.neurons[o].activation_input(w_h2o)
                # ∂C/∂w = (∂C/∂y) * (∂y/∂z) * [(∂z/∂W)]
                pd_errortotal_weight = pd_C_on_z_OutputLayer[o] * Out_h
                # Δw = η * ∂C/∂w
                self.output_layer.neurons[o].weights[w_h2o] -= self.LEARNING_RATE * pd_errortotal_weight

        # 4. 更新隱含層的權重系数
        for h in range(len(self.hidden_layer.neurons)):
            for w_i2h in range(len(self.hidden_layer.neurons[h].weights)):
                # (∂Net_h / ∂W)
                pd_neth_w = self.hidden_layer.neurons[h].activation_input(w_i2h)
                # ∂C/∂W = ∂C/∂Net_h * (∂Net_h/∂W)
                pd_errortotal_weight = pd_C_on_z_NotOutputLayer[h] * pd_neth_w
                # Δw = η * ∂C/∂W
                self.hidden_layer.neurons[h].weights[w_i2h] -= self.LEARNING_RATE * pd_errortotal_weight

    def train(self, training_inputs, targets_outputs):
        self.feed_forward(training_inputs)
        self.feed_backward(targets_outputs)

    def inspect_net(self):
        print('-Hidden Layer-')
        self.hidden_layer.inspect_NL()
        print('-Output Layer-')
        self.output_layer.inspect_NL()

    def calculate_total_loss(self, training_sets):
        total_error = 0
        for t in range(len(training_sets)):
            training_inputs, expect_outputs = training_sets[t]
            self.feed_forward(training_inputs)
            for o in range(len(expect_outputs)):
                total_error += self.output_layer.neurons[o].calculate_error(expect_outputs[o])
        return total_error

In [4]:
import math, random
if __name__ == "__main__":
    nn = NeuralNetwork(num_inputs=2,
                       num_hidden=2,
                       num_outputs=2,
                       hidden_layer_weights=[0.15, 0.2, 0.25, 0.3],
                       hidden_layer_bias=0.35,
                       output_layer_weights=[0.4, 0.45, 0.5, 0.55],
                       output_layer_bias=0.6)
    print("Initial Neural Network:")
    nn.inspect_net()
    print("\n\n\n")
    
    for i in range(1000):  #epoch times
        nn.train([0.05, 0.1], [0.01, 0.99])
        print('***After Train***')
        nn.inspect_net()
        # after point show 9 digits
        print("Iter No." ,i ,"| total_loss =", round(nn.calculate_total_loss([[[0.05, 0.1], [0.01, 0.99]]]), 9))
        print('======================')

Initial Neural Network:
-Hidden Layer-
 Neuron 0
  Weight: 0.15
  Weight: 0.2
  Bias: 0.35
 Neuron 1
  Weight: 0.25
  Weight: 0.3
  Bias: 0.35
-Output Layer-
 Neuron 0
  Weight: 0.4
  Weight: 0.45
  Bias: 0.6
 Neuron 1
  Weight: 0.5
  Weight: 0.55
  Bias: 0.6




***After Train***
-Hidden Layer-
 Neuron 0
  Weight: 0.1497807161327628
  Weight: 0.19956143226552567
  Bias: 0.35
 Neuron 1
  Weight: 0.24975114363236958
  Weight: 0.29950228726473915
  Bias: 0.35
-Output Layer-
 Neuron 0
  Weight: 0.35891647971788465
  Weight: 0.4086661860762334
  Bias: 0.6
 Neuron 1
  Weight: 0.5113012702387375
  Weight: 0.5613701211079891
  Bias: 0.6
Iter No. 0 | total_loss = 0.291027774
***After Train***
-Hidden Layer-
 Neuron 0
  Weight: 0.1495927177634872
  Weight: 0.19918543552697449
  Bias: 0.35
 Neuron 1
  Weight: 0.2495330148608577
  Weight: 0.29906602972171537
  Bias: 0.35
-Output Layer-
 Neuron 0
  Weight: 0.31735386199923843
  Weight: 0.3668504722463087
  Bias: 0.6
 Neuron 1
  Weight: 0.522397326